откроем файл

In [ ]:
!pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c nlp-getting-started
!unzip \*.zip

импортируем все необходимое

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

#models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import xgboost as xgb

#metrics
from sklearn import metrics
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import mean_squared_error



from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline


from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE



#tokens and co.
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from nltk.corpus import stopwords
nltk.download('stopwords')

from string import punctuation

!pip install razdel
from razdel import tokenize

from gensim.models import Doc2Vec
import gensim
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils
from tqdm import tqdm
import spacy

поработем с данными

data_train = pd.read_csv('/content/train.csv')
data_test = pd.read_csv('/content/test.csv')

In [ ]:
data_train.head()

In [ ]:
data_test.head()

тут я сначала решила дропнуть колонки, которые нам не нужны, потом решила еще добавить ключевое слово к тексту, чтобы был один эмбеддинг (что-то типа конкатенации, только до преобразования в вектор)

In [ ]:
data_train.drop('location', axis=1, inplace=True)
data_train.drop('id', axis=1, inplace=True)
data_train.fillna('', inplace=True )
data_train['text'] = data_train['text'] + ' ' + data_train['keyword']
data_train.drop('keyword', axis=1, inplace=True)

In [ ]:
data_test.drop('location', axis=1, inplace=True)
data_test.fillna('', inplace= True )
data_test['text'] = data_test['text'] + ' ' + data_test['keyword']
data_test.drop('keyword', axis=1, inplace=True)

соберем стоп слова

In [ ]:
noise = stopwords.words('english') + list(punctuation)

функции для токенизации, попробуем две

In [ ]:
def tokenizer(string):#razdel
    return [x.text for x in tokenize(string)]

def tokenize_text(text):#nltk
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

поделим на тест и трейн

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_train.text, data_train.target, test_size=0.1)

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)#здесь хранится матрица эмбеддинг

In [ ]:
print(bow)

In [ ]:
list(vec.vocabulary_.items())[:10]

решаем задачу классификации

In [ ]:
clf = LogisticRegression(solver='liblinear', random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

попробуем установить размер n-gram

In [ ]:
vec = CountVectorizer(ngram_range=(2, 3))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(solver='liblinear', random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))
# стало хуже

это с это tfidf

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(C=10, solver='liblinear', random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))





#тут стало лучше!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1

применим стоп-слова и пунктуацию

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer = word_tokenize, stop_words=noise)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(solver='liblinear', random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))


#еще лучше

поменяем токенизатор на раздел

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=tokenizer, stop_words=noise)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(solver='liblinear', random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

# с токенизатором раздела тоже не очень получается

поменяем модель

SVC

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer = word_tokenize, stop_words=noise)
bow = vec.fit_transform(x_train)
clf = SVC()
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer = word_tokenize, stop_words=noise)
bow = vec.fit_transform(x_train)
clf = SVC(kernel='linear')
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))
#уже можно с этим работать

BaggindClassifier

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer = word_tokenize, stop_words=noise)
bow = vec.fit_transform(x_train)
clf = BaggingClassifier()
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))
#  не очень

RandomForestClassifier

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer = word_tokenize, stop_words=noise)
bow = vec.fit_transform(x_train)
clf = RandomForestClassifier(class_weight='balanced')
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))